<a href="https://colab.research.google.com/github/joshnaviseelam/AI-Based-Cyber-Security-Threat-Prediction/blob/main/SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Write Python code to create a sample iris  log table in SQLite, insert records, and define a helper function to run SQL queries.

In [3]:
# 1. Import libraries
import sqlite3
import pandas as pd
from sklearn.datasets import load_iris




In [4]:
# 2. Load the Iris dataset into a pandas DataFrame
iris = load_iris(as_frame=True)
df_iris = iris.frame
df_iris.rename(columns={
    'sepal length (cm)': 'sepal_length',
    'sepal width (cm)': 'sepal_width',
    'petal length (cm)': 'petal_length',
    'petal width (cm)': 'petal_width',
}, inplace=True)
df_iris['species'] = df_iris['target'].map(dict(enumerate(iris.target_names)))


In [5]:
# 3. Create in-memory SQLite database and log table
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

cursor.execute("""
    CREATE TABLE iris_log (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        sepal_length REAL,
        sepal_width REAL,
        petal_length REAL,
        petal_width REAL,
        species TEXT
    );
""")
conn.commit()

In [6]:
# 4. Insert a few sample records into the log table
sample_logs = df_iris.sample(5, random_state=42)

cursor.executemany("""
    INSERT INTO iris_log (sepal_length, sepal_width, petal_length, petal_width, species)
    VALUES (?, ?, ?, ?, ?);
""", sample_logs[['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']].values.tolist())
conn.commit()

In [7]:
# 5. Helper function to run SQL queries and return pandas DataFrame
def run_query(q: str) -> pd.DataFrame:
    """Execute a SQL query against the iris_log table and return the results as a DataFrame."""
    return pd.read_sql_query(q, conn)


In [8]:
# 6. Example usage of the helper function
print("All log entries:")
print(run_query("SELECT * FROM iris_log;"))

print("\nCount of entries per species:")
print(run_query("""
    SELECT species, COUNT(*) AS count
    FROM iris_log
    GROUP BY species;
"""))

All log entries:
   id  sepal_length  sepal_width  petal_length  petal_width     species
0   1           6.1          2.8           4.7          1.2  versicolor
1   2           5.7          3.8           1.7          0.3      setosa
2   3           7.7          2.6           6.9          2.3   virginica
3   4           6.0          2.9           4.5          1.5  versicolor
4   5           6.8          2.8           4.8          1.4  versicolor

Count of entries per species:
      species  count
0      setosa      1
1  versicolor      3
2   virginica      1


Write SQL query to remove duplicate cybersecurity log entries.

In [ ]:
conn.execute("""DELETE FROM iris_log
WHERE id NOT IN (
  SELECT MIN(id)
  FROM iris_log
  GROUP BY sepal_length, sepal_width, petal_length, petal_width, species
);
""")
run_query("SELECT * FROM iris_log")



,id,sepal_length,sepal_width,petal_length,petal_width,species
0,1,6.1,2.8,4.7,1.2,versicolor
1,2,5.7,3.8,1.7,0.3,setosa
2,3,7.7,2.6,6.9,2.3,virginica
3,4,6.0,2.9,4.5,1.5,versicolor
4,5,6.8,2.8,4.8,1.4,versicolor


counting of species

In [13]:
run_query("""
SELECT species, COUNT(*) AS count
FROM iris_log
GROUP BY species;
""")



,species,count
0,setosa,1
1,versicolor,3
2,virginica,1


Species sorted by overall average of all 4 features.

In [14]:
run_query("""SELECT species,
       AVG(sepal_length + sepal_width + petal_length + petal_width) AS avg_all
FROM iris_log
GROUP BY species
ORDER BY avg_all DESC;
""")

,species,avg_all
0,virginica,19.500000
1,versicolor,15.166667
2,setosa,11.500000


Count of flowers where sepal_length BETWEEN 5.0 AND 6.0.

In [15]:
run_query("""SELECT COUNT(*) FROM iris_log WHERE sepal_length BETWEEN 5.0 AND 6.0;
""")


,COUNT(*)
0,2


Get rows where sepal length is above dataset average.

In [16]:
run_query("""SELECT * FROM iris_log
WHERE sepal_length > (SELECT AVG(sepal_length) FROM iris_log);
""")

,id,sepal_length,sepal_width,petal_length,petal_width,species
0,3,7.7,2.6,6.9,2.3,virginica
1,5,6.8,2.8,4.8,1.4,versicolor


Average sepal width grouped by species & rounded petal length.

In [17]:
run_query("""SELECT species, ROUND(petal_length) AS pl_group, AVG(sepal_width)
FROM iris_log
GROUP BY species, pl_group
ORDER BY species, pl_group;
""")

,species,pl_group,AVG(sepal_width)
0,setosa,2.0,3.800000
1,versicolor,5.0,2.833333
2,virginica,7.0,2.600000


Find duplicate rows (before removing).

In [18]:
run_query("""SELECT sepal_length, sepal_width, petal_length, petal_width, species, COUNT(*)
FROM iris_log
GROUP BY sepal_length, sepal_width, petal_length, petal_width, species
HAVING COUNT(*) > 1;
""")

,sepal_length,sepal_width,petal_length,petal_width,species,COUNT(*)


Species with highest minimum sepal length.

In [19]:
run_query("""SELECT species, MIN(sepal_length) AS min_sl
FROM iris_lo"g
GROUP BY species
ORDER BY min_sl DESC
LIMIT 1;
""")

,species,min_sl
0,virginica,7.7


Top 5 longest petals.

In [20]:
run_query("""SELECT * FROM iris_log ORDER BY petal_length DESC LIMIT 5;
""")

,id,sepal_length,sepal_width,petal_length,petal_width,species
0,3,7.7,2.6,6.9,2.3,virginica
1,5,6.8,2.8,4.8,1.4,versicolor
2,1,6.1,2.8,4.7,1.2,versicolor
3,4,6.0,2.9,4.5,1.5,versicolor
4,2,5.7,3.8,1.7,0.3,setosa


Average ratio of petal_length to sepal_length by species.

In [21]:
run_query("""SELECT species, AVG(petal_length / sepal_length) AS avg_ratio
FROM iris_log
GROUP BY species;
""")

,species,avg_ratio
0,setosa,0.298246
1,versicolor,0.742125
2,virginica,0.896104


Group flowers by rounded sepal length (to nearest integer).

In [23]:
run_query("""SELECT ROUND(sepal_length) AS sl_group, COUNT(*)
FROM iris_log
GROUP BY sl_group
ORDER BY sl_group;

""")

,sl_group,COUNT(*)
0,6.0,3
1,7.0,1
2,8.0,1


Count flowers where petal_length > sepal_length.

In [24]:
run_query("""SELECT COUNT(*) FROM iris_log WHERE petal_length > sepal_length;
""")

,COUNT(*)
0,0


Records with sepal width greater than 4.

In [25]:
run_query("""SELECT * FROM iris_log WHERE sepal_width > 4;
""")

,id,sepal_length,sepal_width,petal_length,petal_width,species


Overall average of all four features

In [26]:
run_query("""SELECT AVG(sepal_length) AS avg_sl,
       AVG(sepal_width) AS avg_sw,
       AVG(petal_length) AS avg_pl,
       AVG(petal_width) AS avg_pw
FROM iris_log;
""")

,avg_sl,avg_sw,avg_pl,avg_pw
0,6.46,2.98,4.52,1.34


Species with largest average petal length.

In [27]:
run_query("""SELECT species, AVG(petal_length) AS avg_pl
FROM iris_log
GROUP BY species
ORDER BY avg_pl DESC
LIMIT 1;
""")

,species,avg_pl
0,virginica,6.9
